# Import libraries

In [2]:
import pandas as pd

# Read datasets

In [3]:
i_data = pd.read_excel('dataset/i_data.xlsx')
ui_data = pd.read_excel('dataset/ui_data.xlsx')

# Evaluation

In [4]:
from src.isne_recommendation import FeatureRatingsKNN

In [5]:
model = FeatureRatingsKNN.fit(ui_data)
prediction = FeatureRatingsKNN.predict('Martha Long', ui_data, model, 10)
evaluation = FeatureRatingsKNN.evaluate_model(ui_data, model, 10)

In [9]:
evaluation[evaluation['username'] == 'Martha Long']

,username,education,age,email,address,course,time,payment
471,Martha Long,Middle school level,16,gamil.com,NaN,Build Basic Generative Adversarial Networks (G...,2566-01-01 08:30:00,success
472,Martha Long,Middle school level,16,gamil.com,NaN,Non-Equilibrium Applications of Statistical Th...,2566-03-11 14:01:00,success


### Results 

- **The first approach**: _TF-IDF_ and _Linear Kernel_ Performance has an accuracy score of 7.08 percentage
- **The second approach**: _Feature Ratings_ and _KNN Performance_ has an accuracy score of 21.77 percentage
- **The third approach**: _Hybrid_ has an accuracy score of 27.5 percentage